In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import molpy as mp
from pathlib import Path

data_path = Path("data/case1")

In [ ]:
struct_h = mp.AtomicStructure.from_frame(mp.io.read_pdb(data_path / "H.pdb"), "H")
struct_n = mp.AtomicStructure.from_frame(mp.io.read_pdb(data_path / "N.pdb"), "N")
struct_m = mp.AtomicStructure.from_frame(mp.io.read_pdb(data_path / "M.pdb"), "M")
struct_p = mp.AtomicStructure.from_frame(mp.io.read_pdb(data_path / "P.pdb"), "P")
struct_t = mp.AtomicStructure.from_frame(mp.io.read_pdb(data_path / "T.pdb"), "T")
struct_tfsi = mp.AtomicStructure.from_frame(mp.io.read_pdb(data_path / "TFSI.pdb"), "TFSI")

<Struct: 22 atoms>

In [ ]:
poly_builer = mp.builder.AmberToolsPolymerBuilder(workdir="amber")

In [ ]:
salt_builder = mp.builder.AmberToolsSaltBuilder(workdir="amber")
struct_litfsi = salt_builder.build(
    "LiTFSI",
    [struct_tfsi],
    ["Li+"],
)

<Atom H6>

TAIL_ATOM     15   C8
MAIN_CHAIN     1   15   C8
OMIT_ATOM      1   16   H6
Number of mainchain atoms (including head and tail atom):     1
Number of omited atoms:     1<Atom H4>

HEAD_ATOM      1    C
MAIN_CHAIN     1    1    C
OMIT_ATOM      1    9   H4
Number of mainchain atoms (including head and tail atom):     1
Number of omited atoms:     1

<Struct: 22 atoms>